In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Carregar os arquivos .xlsx
clima_df = pd.read_excel('clima_interlagos_2023.xlsx')
energia_df = pd.read_excel('interrupcao_energia_so_parelheiros.xlsx')

In [ ]:
# Converter a coluna 'Hora (UTC)' em formato de tempo (exemplo: 100 -> 01:00:00)
clima_df['Hora (UTC)'] = clima_df['Hora (UTC)'].apply(lambda x: f"{int(x):04d}")
clima_df['Hora (UTC)'] = pd.to_datetime(clima_df['Hora (UTC)'], format='%H%M').dt.time

# Converter a coluna 'Data' em formato de data, removendo possíveis aspas extras
clima_df['Data'] = pd.to_datetime(clima_df['Data'].str.replace('"', ''), format='%d/%m/%Y')

In [ ]:
# Converter as colunas de tempo na tabela de quedas de energia
energia_df['DatInicioInterrupcao'] = pd.to_datetime(energia_df['DatInicioInterrupcao'])

# Extrair apenas a data e arredondar a hora das interrupções
energia_df['Data'] = energia_df['DatInicioInterrupcao'].dt.date
energia_df['Hora'] = energia_df['DatInicioInterrupcao'].dt.hour + np.round(energia_df['DatInicioInterrupcao'].dt.minute / 60)

# Certificar que a coluna 'Hora' tenha valores inteiros
energia_df['Hora'] = energia_df['Hora'].astype(int)

# Agrupar as interrupções por Data e Hora, e contar o número de ocorrências
energia_agrupada = energia_df.groupby(['Data', 'Hora']).size().reset_index(name='num_ocorrencias')

# Ensure the 'Data' column in energia_agrupada is of datetime type
energia_agrupada['Data'] = pd.to_datetime(energia_agrupada['Data'])

In [ ]:


# Realizar o merge das duas tabelas, baseando-se na tabela do clima (left join) e ignorando a coluna 'Radiacao (KJ/m²)'
merged_df = pd.merge(clima_df, energia_agrupada,
                     left_on=['Data', clima_df['Hora (UTC)'].apply(lambda x: x.hour)],
                     right_on=['Data', 'Hora'],
                     how='left')

# Substituir valores NaN em 'num_ocorrencias' por 0, indicando que não houve queda de energia
merged_df['num_ocorrencias'].fillna(0, inplace=True)

# Substituir valores vazios em 'Chuva (mm)' por 0
merged_df['Chuva (mm)'].fillna(0, inplace=True)


# Selecionar as colunas desejadas (excluindo 'Radiacao (KJ/m²)')
resultado_df = merged_df[['Data', 'Hora (UTC)', 'Temp. Ins. (C)', 'Umi. Ins. (%)', 'Vel. Vento (m/s)',
                          'Raj. Vento (m/s)', 'Chuva (mm)', 'num_ocorrencias']]
# Remover as colunas "Data" e "Hora (UTC)"
resultado_df = resultado_df.drop(['Data', 'Hora (UTC)'], axis=1) 

#transformar os ; em Nan
resultado_df.replace(';', np.nan, inplace=True)
resultado_df.dropna(inplace=True)

#remover Nan
resultado_df.dropna(inplace=True)

if 4424 in resultado_df.index:
  # Se o índice existir, substituir o valor na coluna 'Chuva (mm)' por 0
  resultado_df.loc[4424, 'Chuva (mm)'] = 0

if 4443 in resultado_df.index:
  # Se o índice existir, substituir o valor na coluna 'Chuva (mm)' por 0
  resultado_df.loc[4443, 'Chuva (mm)'] = 0

# Alterar o valor na linha 6089 da coluna 'Umi. Ins. (%)' para 15
if 6089 in resultado_df.index:
  resultado_df.loc[6089, 'Umi. Ins. (%)'] = 15

# Alterar o valor na linha 6159 da coluna 'Umi. Ins. (%)' para 79
if 6159 in resultado_df.index:
  resultado_df.loc[6159, 'Umi. Ins. (%)'] = 79

# Converter a coluna 'Chuva (mm)' para numérico
resultado_df['Chuva (mm)'] = pd.to_numeric(resultado_df['Chuva (mm)'], errors='coerce')

# Converter a coluna 'Umi. Ins. (%)' para numérico
if not pd.api.types.is_string_dtype(resultado_df['Umi. Ins. (%)']):
  resultado_df['Umi. Ins. (%)'] = resultado_df['Umi. Ins. (%)'].astype(str).str.replace(',', '.').astype(float)
else:
  resultado_df['Umi. Ins. (%)'] = resultado_df['Umi. Ins. (%)'].str.replace(',', '.').astype(float)
# Converter a coluna 'Chuva (mm)' para numérico
resultado_df['Chuva (mm)'] = pd.to_numeric(resultado_df['Chuva (mm)'], errors='coerce')

# Salvar o DataFrame em um arquivo CSV
resultado_df.to_csv('resultado.csv', index=False)